In [ ]:
!pip install transformers datasets

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset_narasi_kebijakan_sentimen_publik.txt to dataset_narasi_kebijakan_sentimen_publik.txt


In [ ]:
# STEP 3: Load tokenizer dan model GPT-2
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

# STEP 4: Load dataset (ubah nama sesuai file kamu)
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="dataset_narasi_kebijakan_sentimen_publik.txt",  # ← ubah kalau nama beda
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-kebijakan-output",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=True,
    report_to="none"  # 🔥 ini kunci mematikan wandb
)


In [ ]:
# STEP 6: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# STEP 7: Train
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.507900
200,0.550800
300,0.291100
400,0.184800
500,0.150700
600,0.129500
700,0.112400
800,0.114500


TrainOutput(global_step=861, training_loss=0.47705690979819126, metrics={'train_runtime': 44.0455, 'train_samples_per_second': 39.028, 'train_steps_per_second': 19.548, 'total_flos': 112290250752000.0, 'train_loss': 0.47705690979819126, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./gpt2_sentimen")
tokenizer.save_pretrained("./gpt2_sentimen")

('./gpt2_sentimen/tokenizer_config.json',
 './gpt2_sentimen/special_tokens_map.json',
 './gpt2_sentimen/vocab.json',
 './gpt2_sentimen/merges.txt',
 './gpt2_sentimen/added_tokens.json')

In [ ]:
inputs = tokenizer.encode_plus(HasilPrediksi, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"].to("cuda")
attention_mask = inputs["attention_mask"].to("cuda")

output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=300,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

print("🟦 PROMPT:\n", HasilPrediksi)
print("🟩 REKOMENDASI:\n", tokenizer.decode(output[0], skip_special_tokens=True))
print("=" * 100)

🟦 PROMPT:
 Prediksi harga untuk 2025-06-15: Rp21,723

Berdasarkan hasil prediksi model, harga beras menunjukkan tren kenaikan selama 240 hari ke depan, dari Rp21,525 menjadi Rp27,635, atau perubahan sekitar 28.39%. Perubahan paling signifikan terjadi pada 2025-12-03 dengan selisih sekitar Rp1,442.
🟩 REKOMENDASI:
 Prediksi harga untuk 2025-06-15: Rp21,723

Berdasarkan hasil prediksi model, harga beras menunjukkan tren kenaikan selama 240 hari ke depan, dari Rp21,525 menjadi Rp27,635, atau perubahan sekitar 28.39%. Perubahan paling signifikan terjadi pada 2025-12-03 dengan selisih sekitar Rp1,442.
Rekomendasi: Pemerintah perlu segera menggelar operasi pasar untuk menekan lonjakan harga.

Kesimpulan: Berdasarkan hasil prediksi model, harga beras menunjukkan tren naik selama 8 hari ke depan, dari Rp12,921 menjadi Rp12,913, atau perubahan sekitar 4.17%. Perubahan paling signifikan terjadi pada 2025-12-12 dengan selisih sekitar Rp442.
Rekomendasi: Pemerintah perlu segera menggelar operasi pa

In [ ]:
# Pastikan model dan tokenizer sudah loaded, dan model ke CUDA
model.to("cuda")

prompts = [
    "Kesimpulan: Berdasarkan hasil prediksi model, harga beras menunjukkan tren naik selama 10 hari ke depan, dari Rp13.000 menjadi Rp13.800, atau perubahan sekitar 6.15%. Perubahan paling signifikan terjadi pada 2025-06-12 dengan selisih sekitar Rp500.\nRekomendasi:",
    "Kesimpulan: Berdasarkan hasil prediksi model, harga beras turun perlahan selama 14 hari ke depan, dari Rp12.700 menjadi Rp12.200, atau perubahan sekitar -3.94%. Puncak penurunan terjadi pada 2025-06-08.\nRekomendasi:",
    "Kesimpulan: Harga beras diprediksi stabil selama 7 hari ke depan, fluktuasi kurang dari 0.5% antara Rp12.400 dan Rp12.450.\nRekomendasi:",
    "Kesimpulan: Prediksi menunjukkan lonjakan harga beras dari Rp13.500 ke Rp14.500 dalam 5 hari (naik 7.4%).\nRekomendasi:",
    "Kesimpulan: Selama 12 hari ke depan, harga beras mengalami penurunan tajam dari Rp12.800 menjadi Rp11.900 (-7%).\nRekomendasi:"
]

for prompt in prompts:
    inputs = tokenizer.encode_plus(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=300,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    print("🟦 PROMPT:\n", prompt)
    print("🟩 REKOMENDASI:\n", tokenizer.decode(output[0], skip_special_tokens=True))
    print("=" * 100)

🟦 PROMPT:
 Kesimpulan: Berdasarkan hasil prediksi model, harga beras menunjukkan tren naik selama 10 hari ke depan, dari Rp13.000 menjadi Rp13.800, atau perubahan sekitar 6.15%. Perubahan paling signifikan terjadi pada 2025-06-12 dengan selisih sekitar Rp500.
Rekomendasi:
🟩 REKOMENDASI:
 Kesimpulan: Berdasarkan hasil prediksi model, harga beras menunjukkan tren naik selama 10 hari ke depan, dari Rp13.000 menjadi Rp13.800, atau perubahan sekitar 6.15%. Perubahan paling signifikan terjadi pada 2025-06-12 dengan selisih sekitar Rp500.
Rekomendasi: Pertahankan kualitas pelayanan medis dan perluas cakupan program seperti BPJS dan vaksinasi. Lakukan dokumentasi praktik baik sebagai model untuk wilayah lain.

Kesimpulan: Pandangan masyarakat terhadap sektor layanan perpustakaan bersifat netral. Sebagian besar tanggapan menunjukkan sikap yang tidak condong kuat ke arah positif atau negatif, seperti: Pengunjung merasa layanan berjalan seperti biasa biasa.

Kesimpulan: Pandangan masyarakat terha

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load model dan tokenizer dari folder hasil fine-tuning
model = GPT2LMHeadModel.from_pretrained("./gpt2_beras")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_beras")

import torch

input_ids = tokenizer.encode(HasilPrediksi, return_tensors="pt")

# Generate teks
output_ids = model.generate(
    input_ids,
    max_length=1000,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.eos_token_id  # penting agar tidak error saat padding
)

# Decode hasil
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Prediksi harga untuk 2025-06-15: Rp21,723

Berdasarkan hasil prediksi model, harga beras menunjukkan tren kenaikan selama 240 hari ke depan, dari Rp21,525 menjadi Rp27,635, atau perubahan sekitar 28.39%. Perubahan paling signifikan terjadi pada 2025-12-03 dengan selisih sekitar Rp1,442.
Rekomendasi: Pemerintah dapat menyerap kelebihan pasokan untok memperkuat cadangan pangan nasional. Perluasan distribusi Bulog dapskipun harpa 8 haris kembali stok berasi stabilkan harsi.


In [ ]:
import shutil

# Path ke folder di Colab
source_folder = "/content/gpt2_sentimen"
zip_path = "/content/gpt2_sentimen.zip"

# Buat file ZIP
shutil.make_archive("/content/gpt2_sentimen", 'zip', source_folder)

print("ZIP berhasil dibuat di:", zip_path)

ZIP berhasil dibuat di: /content/gpt2_sentimen.zip
